<a href="https://colab.research.google.com/github/beza-lab/KAIM-WEEK-5/blob/main/task3_fine_tune_NER_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
from google.colab import files
uploaded = files.upload()

Saving dataset.conll to dataset (1).conll


In [6]:
import pandas as pd

# Reading the .conll file into a DataFrame
dataset_path = '/content/dataset (1).conll'
# Make sure this file path is correct and the file exists in this path
data = pd.read_csv(dataset_path, sep='\t', header=None, skip_blank_lines=True)

# Check the first few rows of the dataset
print(data.head())


              0
0  ⬇️⬇️⬇️9pcs O
1     Kitchen O
2       knife O
3         Set O
4         ✔️1 O


In [9]:
import re
import pandas as pd
from itertools import groupby

# Load the dataset
dataset_path = '/content/dataset.conll'  # Ensure the file path is correct
data = pd.read_csv(dataset_path, sep='\t', header=None, skip_blank_lines=False, names=['token', 'label'])

# Define a function to clean and parse the dataset
def clean_and_parse_data(data):
    tokens, labels = [], []
    token_seq, label_seq = [], []

    for row in data.itertuples(index=False):
        if pd.isna(row.token):
            if token_seq:  # Append token/label sequence only if it exists
                tokens.append(token_seq)
                labels.append(label_seq)
                token_seq, label_seq = [], []
        else:
            token = re.sub(r'[^A-Za-z0-9ሀ-መ-\s]', '', str(row.token))
            label = row.label if row.label else 'O'
            token_seq.append(token)
            label_seq.append(label)

    # Ensure the last sequence is appended
    if token_seq:
        tokens.append(token_seq)
        labels.append(label_seq)

    return {'tokens': tokens, 'ner_tags': labels}

# Clean and parse dataset
dataset_dict = clean_and_parse_data(data)

# Converting dictionary to Hugging Face Dataset
from datasets import Dataset

dataset = Dataset.from_dict(dataset_dict)

print(dataset)


Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 14
})
